Naive attempt to implement TONAL PITCH STEP DISTANCE:
A SIMILARITY MEASURE FOR CHORD PROGRESSIONS
W. Bas de Haas, Remco C. Veltkamp, Frans Wiering

In [ ]:
import pretty_midi
import music21
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
import numpy as np
import matplotlib.pyplot as plt

file1 = "/content/The Police - Every Breath You Take.mid"
file2 = "/content/I'll Be Missing You.1.mid"

# === TPS Distance Function ===
def tps_distance(chord1_label, chord2_label, key_str):
    if 'GAP' in (chord1_label, chord2_label):
        return 2.0  # Max penalty for GAPs
    try:
        k = music21.key.Key(key_str)
        ch1 = music21.harmony.ChordSymbol(chord1_label)
        ch2 = music21.harmony.ChordSymbol(chord2_label)

        sd1 = k.getScaleDegreeFromPitch(ch1.root())
        sd2 = k.getScaleDegreeFromPitch(ch2.root())
        scale_degree_distance = abs(sd1 - sd2)

        pcs1 = set(p.name for p in ch1.pitches)
        pcs2 = set(p.name for p in ch2.pitches)
        shared_notes = len(pcs1 & pcs2)
        total_notes = len(pcs1 | pcs2)
        overlap_score = shared_notes / total_notes if total_notes > 0 else 0

        return round(scale_degree_distance - 2 * overlap_score, 3)
    except:
        return 2.0

# === Distance Function for DTW and SW ===
def music_distance(a, b, key):
    pitch_diff = abs(a[0] - b[0]) / 12.0  # normalized pitch class distance
    time_diff = abs(a[1] - b[1])          # onset difference in seconds
    chord_diff = tps_distance(a[2], b[2], key)
    return 0.5 * pitch_diff + 0.2 * time_diff + 0.3 * chord_diff

# === Parse MIDI with Chords ===
def extract_note_chord_sequence(midi_path):
    pm = pretty_midi.PrettyMIDI(midi_path)
    m21 = music21.converter.parse(midi_path)
    key_estimate = m21.analyze('key')
    chordified = m21.chordify()

    # Extract chords with their offsets
    chord_map = []
    for c in chordified.recurse().getElementsByClass('Chord'):
        chord_map.append((c.offset, c.commonName or "Unknown"))

    # Extract melodic notes and assign nearest preceding chord
    sequence = []
    for instrument in pm.instruments:
        if instrument.is_drum:
            continue
        for note in instrument.notes:
            raw_chord = next((ch for off, ch in reversed(chord_map) if off <= note.start), "Unknown")
            try:
                _ = music21.harmony.ChordSymbol(raw_chord)
                chord_label = raw_chord
            except:
                chord_label = "GAP"
            sequence.append([note.pitch % 12 + 1, note.start, chord_label])

    return sequence, key_estimate.tonic.name

# === Dynamic Time Warping ===
def run_dtw(seq1, seq2, key):
    vec1 = [[music_distance(a, [0, 0, "GAP"], key)] for a in seq1]
    vec2 = [[music_distance(b, [0, 0, "GAP"], key)] for b in seq2]
    return fastdtw(vec1, vec2, dist=euclidean)

# === Smith-Waterman Algorithm with traceback ===
def smith_waterman(seq1, seq2, key, gap_penalty=2.0):
    m, n = len(seq1), len(seq2)
    H = np.zeros((m+1, n+1))
    traceback = np.zeros((m+1, n+1), dtype=int)
    max_score, max_pos = 0, (0, 0)

    for i in range(1, m+1):
        for j in range(1, n+1):
            match = H[i-1][j-1] + (2 - music_distance(seq1[i-1], seq2[j-1], key))
            delete = H[i-1][j] - gap_penalty
            insert = H[i][j-1] - gap_penalty
            scores = [0, match, delete, insert]
            H[i][j] = max(scores)
            traceback[i][j] = scores.index(H[i][j])
            if H[i][j] >= max_score:
                max_score = H[i][j]
                max_pos = (i, j)

    # Traceback
    aligned_seq1 = []
    aligned_seq2 = []
    i, j = max_pos
    while H[i][j] > 0:
        if traceback[i][j] == 1:
            aligned_seq1.insert(0, seq1[i-1])
            aligned_seq2.insert(0, seq2[j-1])
            i -= 1
            j -= 1
        elif traceback[i][j] == 2:
            i -= 1
        elif traceback[i][j] == 3:
            j -= 1

    return H, max_score, max_pos, aligned_seq1, aligned_seq2

# === Comparison Function ===
def compare_two_midi_files(file1, file2):
    seq1, key1 = extract_note_chord_sequence(file1)
    seq2, key2 = extract_note_chord_sequence(file2)

    dtw_distance, dtw_path = run_dtw(seq1, seq2, key1)
    sw_matrix, sw_score, sw_end, aligned1, aligned2 = smith_waterman(seq1, seq2, key1)

    return {
        'dtw_distance': dtw_distance,
        'dtw_path_len': len(dtw_path),
        'sw_score': sw_score,
        'sw_matrix': sw_matrix.tolist(),
        'aligned_seq1': aligned1,
        'aligned_seq2': aligned2
    }

# Run it and print results
results = compare_two_midi_files(file1, file2)
print("=== Comparison Results ===")
print(f"DTW Distance: {results['dtw_distance']:.3f}")
print(f"DTW Path Length: {results['dtw_path_len']}")
print(f"Smith-Waterman Score: {results['sw_score']:.3f}")

print("\n=== Smith-Waterman Aligned Motif ===")
for a, b in zip(results['aligned_seq1'], results['aligned_seq2']):
    print(f"A: Pitch {a[0]}, Time {a[1]:.2f}, Chord {a[2]} | B: Pitch {b[0]}, Time {b[1]:.2f}, Chord {b[2]}")

Brief DTW for Case Study


In [ ]:
!pip install fastdtw
!pip install scipy

In [ ]:
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
import pretty_midi

def extract_pitch_intervals(notes):
    notes = sorted(notes, key=lambda n: n.start)
    pitches = [note.pitch for note in notes]
    print([pitches[i+1] - pitches[i] for i in range(len(pitches)-1)])
    return [pitches[i+1] - pitches[i] for i in range(len(pitches)-1)]

def get_track_intervals(midi_path, track_index=0):
    midi_data = pretty_midi.PrettyMIDI(midi_path)
    if len(midi_data.instruments) <= track_index:
        raise IndexError("Track index out of range.")
    notes = midi_data.instruments[track_index].notes
    return extract_pitch_intervals(notes)

# === Load tracks and extract pitch interval sequences ===
query_intervals = get_track_intervals("/content/isolated_tracks/I_ll Be Missing You_1_Tracks/Track2_Guitar.mid", track_index=0)
candidate_intervals = get_track_intervals("/content/Track2_EVERY BREATH YOU TAKE           _Words and music by Sting.mid", track_index=0)

# === Run fastdtw alignment ===
distance, path = fastdtw(
    [(x,) for x in query_intervals],
    [(y,) for y in candidate_intervals],
    dist=euclidean
)


print(f"DTW Distance: {distance}")
print(f"DTW Path Length: {len(path)}")

In [ ]:
import matplotlib.pyplot as plt

query = query_intervals
candidate = candidate_intervals
x_path, y_path = zip(*path)

plt.plot(query, label="Query", linewidth=2)
plt.plot(candidate, label="Candidate", linewidth=2)
plt.scatter(x_path, [query[i] for i in x_path], c='r', s=5, label="Aligned Points")
plt.title("Pitch Interval Alignment (DTW)")
plt.legend()
plt.show()